In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import unidecode
import math
import numpy as np

In [2]:
tabla = pd.read_csv('atributos_jugadores_21_22.csv')

In [3]:
r = requests.get('https://es.besoccer.com/competicion/clasificacion/primera/2022') # optener html de la pagina
soup = bs(r.content,'html.parser')
r.status_code

200

In [4]:
# optener con regex el nombre del equipo para completar link de plantilla
equipo_soup = soup.find_all('td',attrs={"class":"name"})
link_equipos=[]
for i in range(0,20):
    a=re.findall(r'[a-z]+[\-]?[a-z]+$',equipo_soup[i].a['href']) 
    link_equipos.append(a)

In [19]:

r = requests.get('https://es.besoccer.com/competicion/clasificacion/primera/2019') # optener html de la pagina
soup = bs(r.content,'html.parser')
r.status_code


# optener con regex el nombre del equipo para completar link de plantilla
equipo_soup = soup.find_all('td',attrs={"class":"name"})
link_equipos=[]
for i in range(0,20):
    a=re.findall(r'[a-z]+[\-]?[a-z]+$',equipo_soup[i].a['href']) 
    link_equipos.append(a)

# plantilla y atributos de cada jugador
nombre = []
equipo = []
posicion = []
ritmo = []
tiro = []
pase = []
regate = []
defensa = []
fisico = []
salto = []
estirada = []
parada = []
saque = []
colocacion = []
reflejo = []
jugadores_sin_atri = []

for i in link_equipos:
  r1 = requests.get(f'https://es.besoccer.com/equipo/plantilla/{i[0]}/2022') # optener html de la pagina
  soup1 = bs(r1.content,'html.parser')
  posicion1 = soup1.find('tbody')

  for j in posicion1.find_all('tr'):
      if j.th: # solo cuando este campo existe se actualiza la variable de la posicion del jugador
          pos=unidecode.unidecode(j.th.text[0:-1]).upper() # convertir posición en mayuscula y sin caracteres especiales
      else:
          nom=unidecode.unidecode(j.find_all('td')[1].img['alt']).upper() # convertir nombre en mayuscula y sin caracteres especiales
          equi = unidecode.unidecode(soup1.find('h2',attrs={'class':'title ta-c'}).text).upper() # equipo del jugador
          url_jug = j.find_all('td')[1].a['href'] # linnk del jugador

          r_1 = requests.get(url_jug) # optener html de la pagina del jugador
          soup_1 = bs(r_1.content,'html.parser')
          atri = soup_1.find_all('div',attrs={"class":"cvalue"})
          atri = [i.text for i in atri] # crear lista con atributos del jugador

          if atri: # verifico si el jugador tiene atributos
            if pos=='PORTERO':
              for i,j in enumerate(atri):
                if i==0: 
                  if j: salto.append(j) # if para verificar que el atributo existe , si no existe agregar cero.
                  else: salto.append(0)
                if i==1: 
                  if j: estirada.append(j)
                  else: estirada.append(0)
                if i==2: 
                  if j:parada.append(j)
                  else: parada.append(0)
                if i==3: 
                  if j:saque.append(j)
                  else: saque.append(0)
                if i==4: 
                  if j:colocacion.append(j)
                  else: colocacion.append(0)
                if i==5: 
                  if j:reflejo.append(j) 
                  else: reflejo.append(0)
              nombre.append(nom)
              equipo.append(equi)
              posicion.append(pos)
              ritmo.append(0)
              tiro.append(0)
              pase.append(0)
              regate.append(0)
              defensa.append(0)
              fisico.append(0)
            else:
              for i,j in enumerate(atri):
                if i==0: 
                  if j:ritmo.append(j)
                  else: ritmo.append(0)
                if i==1: 
                  if j:tiro.append(j)
                  else: tiro.append(0)
                if i==2: 
                  if j:pase.append(j)
                  else: pase.append(0)
                if i==3: 
                  if j:regate.append(j)
                  else: regate.append(0)
                if i==4: 
                  if j:defensa.append(j)
                  else: defensa.append(0)
                if i==5: 
                  if j:fisico.append(j)
                  else: fisico.append(0)
              nombre.append(nom)
              equipo.append(equi)
              posicion.append(pos)
              salto.append(0)
              estirada.append(0)
              parada.append(0)
              saque.append(0)
              colocacion.append(0)
              reflejo.append(0) 
          else: # si no tiene atributos lo creo con todos los atributos en cero
            nombre.append(nom)
            jugadores_sin_atri.append(nom)
            equipo.append(equi)
            posicion.append(pos)
            ritmo.append(0)
            tiro.append(0)
            pase.append(0)
            regate.append(0)
            defensa.append(0)
            fisico.append(0)
            salto.append(0)
            estirada.append(0)
            parada.append(0)
            saque.append(0)
            colocacion.append(0)
            reflejo.append(0) 

In [6]:
jugadores_21_22 = pd.DataFrame({"nombre":nombre,"equipo":equipo,"posicion":posicion,"ritmo":ritmo,"tiro":tiro,
                            "pase":pase,"regate":regate,"defensa":defensa,"fisico":fisico,"salto":salto,"estirada":estirada,"parada":parada,"saque":saque,"colocacion":colocacion,"reflejos":reflejo})

In [7]:
# cambio el tipo de dato a int de las columnas numericas
jugadores_21_22 = jugadores_21_22.astype({"ritmo":"int","tiro":"int", "pase":"int","regate":"int","defensa":"int","fisico":"int","salto":"int","estirada":"int","parada":"int","saque":"int","colocacion":"int","reflejos":"int"})

In [8]:
# remplazo los ceros pos NaN  para poder operar y que los ceros no afecten las operaciones como la media.
jugadores_21_22 = jugadores_21_22.replace(0, np.NaN)

In [9]:
# creamos copia para completar los jugadores que no tiene atributos
copia=jugadores_21_22.copy()

In [10]:
posi = copia.loc[:,'posicion'].unique()
equipos = copia.loc[:,'equipo'].unique()

In [11]:
# funcion para remplazar los NaN de los jugadores sin atributos por la media de sus compañeros de la misma posicion
def media_atri_NaN(x):
    if x.dtype!=jugadores_21_22['nombre'].dtype:
        return x.fillna(x.mean())
    return x

In [12]:
# Inicializamos variable para poder hacer un acumulador
copia2=copia[copia['nombre']=='luis']
copia2

,nombre,equipo,posicion,ritmo,tiro,pase,regate,defensa,fisico,salto,estirada,parada,saque,colocacion,reflejos


In [13]:
for i in equipos:
    for j in posi:
        copia1=copia[(copia['posicion']==j)&(copia['equipo']==i)].apply(media_atri_NaN)
        copia2=pd.concat([copia2, copia1],axis=0,ignore_index=True)

In [14]:
jugadores_21_22[jugadores_21_22['equipo']=='VALENCIA'].head()

,nombre,equipo,posicion,ritmo,tiro,pase,regate,defensa,fisico,salto,estirada,parada,saque,colocacion,reflejos
208,JASPER CILLESSEN,VALENCIA,PORTERO,NaN,NaN,NaN,NaN,NaN,NaN,71.0,82.0,81.0,78.0,81.0,81.0
209,G. MAMARDASHVILI,VALENCIA,PORTERO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,JAUME DOMENECH,VALENCIA,PORTERO,NaN,NaN,NaN,NaN,NaN,NaN,72.0,73.0,77.0,75.0,71.0,75.0
211,J. GAYA,VALENCIA,DEFENSA,83.0,62.0,78.0,81.0,67.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN
212,GABRIEL PAULISTA,VALENCIA,DEFENSA,69.0,52.0,69.0,66.0,61.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
copia2[copia2['equipo']=='VALENCIA'].head()

,nombre,equipo,posicion,ritmo,tiro,pase,regate,defensa,fisico,salto,estirada,parada,saque,colocacion,reflejos
208,JASPER CILLESSEN,VALENCIA,PORTERO,NaN,NaN,NaN,NaN,NaN,NaN,71.0,82.0,81.0,78.0,81.0,81.0
209,G. MAMARDASHVILI,VALENCIA,PORTERO,NaN,NaN,NaN,NaN,NaN,NaN,71.5,77.5,79.0,76.5,76.0,78.0
210,JAUME DOMENECH,VALENCIA,PORTERO,NaN,NaN,NaN,NaN,NaN,NaN,72.0,73.0,77.0,75.0,71.0,75.0
211,J. GAYA,VALENCIA,DEFENSA,83.0,62.0,78.0,81.0,67.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN
212,GABRIEL PAULISTA,VALENCIA,DEFENSA,69.0,52.0,69.0,66.0,61.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#copia2.to_csv('atributos_jugadores_2019.csv')